In [1]:
from threading import Thread
from queue import Queue
import sys
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
import pickle
from sklearn.metrics import f1_score

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import random
import time

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score, auc, confusion_matrix, accuracy_score

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import tree

In [2]:
HOME_PATH = '/home/joaomcouto/git/E02_misinformation_detection/'
MODELS_PATH = HOME_PATH + 'MM/Models/'
TABLES_PATH = HOME_PATH
GRAPHS_PATH = HOME_PATH + 'MM/Graphs/'
IMAGES_PATH = HOME_PATH + 'MM/Clusters/'

#N_FOLDS = 5
N_FOLDS = 5
RANDOM_STATE = 1

TASK_NAME = 'desinformacao_urls'

In [3]:
#Recebe features e gera x combinações de tamanho r
def random_combinations(iterable, r, x, seed=10):
    #r é o tamanho dos modelos a serem gerados
    #x é o numero de modelos a serem gerados
    #iterable é o conjunto de features sobre as quais cada modelo sera definidos
    #ps: aqui, um modelo é o conjunto de features contida nele
    pool = tuple(iterable) #Transforma o conjunto de features numa tupla
    n = len(pool) #n é o numero de features 
    a = [] #a vai ser uma lista de listas onde cada elemento é um modelo gerado
    random.seed(seed) 
    for i in range(x): #Vai gerar um modelo x vezes
        indices = sorted(random.sample(range(n), r)) #Seleciona r indices aleatorios do conjunto de features
        a.insert(len(a), tuple(pool[i] for i in indices)) #Insere as features desses indices, em a, como um tupla
    return list(set(a)) #Ao final "a" tem até x modelos unicos com r features cada

In [4]:
#Recebe uma lista de de modelos de mesmo tamanho encontra melhores hiperparametros para esse tamanho
#Os hiperparametros encontrados são salvos em um arquivo
#Isso é calculado treinando todos os modelos em comb com cada combinação de hipeparamentro e,
# pegando a combinação de maior F1 (antes era AUC)
def gridSearch(df, comb, c):
    #comb é uma lista de tuplas onde cada uma é um modelo (conjunto de features)
    #df é o dataframe com os dados (features e label) das instancias
    #c é o tamanho dos modelos em comb
    bestParams = {'random_state': 20200225, 'criterion': 'gini', 'max_depth': None,
                  'min_samples_split': 71, 'min_samples_leaf': 29, 'min_impurity_decrease': 0.0}
    bestAUC = -1

    mdrange = [None, 3, 5, 10]
    criterions = ['gini', 'entropy']
    mssrange = list(range(5, 101, 5))
    mslrange = list(range(5, 51, 5))
    midrange = [0.0, 0.01, 0.1]
    
    #numero total de combinações de parametros que serão testados COM CADA modelo
    combinations = len(mdrange)*len(mssrange)*len(mslrange)*len(midrange)*len(criterions)

    #Verifica se o gridsearch ja foi feito pra esse tamanho de modelo (salvo em $TASK_NAME$-size%TAMANHO%-gridsearch.pkl)
    #Se for, 
    #coloca os parametros na lista griddone onde cada posição indice i é bestParams pra modelos tamanho i e,
    #ja retorna o melhor conjunto de parametros.
    if os.path.isfile(MODELS_PATH + 'MultipleModels_DecisionTrees/' + TASK_NAME + '-size%d-gridsearch.pkl' % c):
        with open(MODELS_PATH + 'MultipleModels_DecisionTrees/' + TASK_NAME + '-size%d-gridsearch.pkl' % c, 'rb') as pkldic:
            bestParams = pickle.load(pkldic)
            griddone[c] = combinations
        return bestParams

    tdone = 0.0
    begin = time.time()
    z = 0
    for crit in criterions:
        for md in mdrange:
            for mss in mssrange:
                for msl in mslrange:
                    for mid in midrange:
                        tdone += 1
                        aucs = []
                        # ff eh um modelo pq comb eh uma lista de tuplas onde cada tupla é um modelo(conjunto de features) de mesmo tamanho
                        for ff in comb:
                            f = []
                            for x in ff:  # transforma a tupla com o modelo em uma lista (f)
                                f.insert(len(f), x)
                            auc, accmedia, preds, probs,f1,_ = select_features_platelabel(df, f,{'random_state': 1, 'max_depth': md, 'min_samples_split': mss, 'min_samples_leaf': msl, 'min_impurity_decrease': mid, 'criterion': crit}, nfolds=4,f1i=True)
                            aucs.extend(auc)
                            z += 1
                            # print(z,len(comb))

                        if np.mean(f1) > bestAUC:
                            bestParams['max_depth'] = md
                            bestParams['min_samples_split'] = mss
                            bestParams['min_samples_leaf'] = msl
                            bestParams['min_impurity_decrease'] = mid
                            bestParams['criterion'] = crit
                            bestAUC = np.mean(f1)

                        if c != 0:
                            now = time.time()
                            elapsed = now-begin
                            perinstance = float(elapsed)/float(tdone)
                            predicted = perinstance * combinations
                            griddone[c] += 1
                            sys.stdout.write('GridSearch (size %02d) Progress: %.3f%% (%d/%d) [Elapsed: %ds | Predicted %ds | Avg: %ds]\r' % (
                                c, 100.0*tdone/combinations, tdone, combinations, elapsed, predicted, perinstance))
                            sys.stdout.flush()

    with open(MODELS_PATH + 'MultipleModels_DecisionTrees/' + TASK_NAME + '-size%d-gridsearch.pkl' % c, 'wb') as pkldic:
        pickle.dump(bestParams, pkldic)
    return(bestParams)


In [5]:
# Recebe varios modelos DE MESMO TAMANHO, cada uma é uma tupla em comb
#Efetiva ou carrega o gridsearch para o tamanho, classifica, salva resultados+preds e,
# retorna uma lista com as aucs medias dos modelos daquele tamanho
def eval_panel_platelabel(df, comb, c, exit_stat, exit_outp):
    #df dado
    #comb lista com modelos com c features
    #exit_stat path para o csv onde resultados serão guardados 
    #exit_outp path para o csv onde predições serão guardadas 
    
    #fpath parece ser a mesma coisa que exit_outp
    fpath = MODELS_PATH + 'MultipleModels_DecisionTrees/' + TASK_NAME + '-size%d-preds.csv' % c

    performed = [] #Uma lista com todas as combinações de features (modelos) ja efetuados
    
    #Verifica se o csv com as predições ja foi criado e tem conteudo, isso indica uma execução parcial previa para esse tamanho)
    #Sendo o caso, carrega as combinações de features ja exploradas na lista performed
    if os.path.isfile(fpath) and os.path.getsize(fpath) > 0:
        performed = list(pd.read_csv(
            fpath, delimiter=';', header=0)['features'])
        
        #done[i] armazena o número de combinações(modelos) exploradas com i features
        done[c] += len(performed)
        global predone
        #predone armazena o número total de combinações que já haviam sido exploradas (encontradas no csv)
        predone += len(performed)
        
        #assegura que os proximos resultados e predições serão appendados numa nova linha
        exit_outp.write('\n')
        exit_stat.write('\n')
        
    #Não sendo o caso estruturamos os headers (primeira linha)do csv para receber as predições e probabilidades,
    #de cada instancia no dado (df)
    else:
        exit_outp.write('features')
        for i in range(len(df)):
            exit_outp.write(';pred%d' % (i+1))
        for i in range(len(df)):
            exit_outp.write(';prob%d' % (i+1))
        exit_outp.write('\n')

    #Faz o gridsearch usando até 50 modelos daquele tamanho
    params = gridSearch(df, comb[:max(50, int(0.001*float(len(comb))))], c)

    ncomb = [] #Sera uma list com os modelos em comb a menos daqueles que já foram explorados (estao em performed)
    begin = time.time()
    tdone = 0.0 #Armazena o total de novas combinações exploradas (não estavam em performed e foram efetivadas)

    for ff in comb:
        f = []
        for x in ff:  # transforma a tupla com o modelo em uma lista só pra conseguir ver se ta no performed
            f.insert(len(f), x)

        if str(f) not in performed:
            ncomb.append(ff)
            
    comb = ncomb
    res = []
    # ff eh um modelo pois comb eh uma lista de tuplas onde cada tupla é um modelo(conjunto de features) de mesmo tamanho
    #lembrando que comb foi atualizado para conter apenas as combinações que já não estavam em performed
    for ff in comb:
        tdone += 1
        now = time.time()
        elapsed = now-begin
        perinstance = float(elapsed)/float(tdone) #Tempo médio gasto em cada combinação
        predicted = perinstance * len(comb) #Tempo estimado até o fim de todas as combinações DESSE TAMANHO
        sys.stdout.write('MM (size %02d) Progress: %.3f%% (%d/%d) [Elapsed: %ds | Predicted %ds | Avg: %ds]\r' % (
            c, 100.0*tdone/len(comb), tdone, len(comb), elapsed, predicted, perinstance))
        sys.stdout.flush()

        global s #Variavel global que define o número maximos de modelos que exploraremos pra um tamanho
                    #Até segunda ordem setado em 10000
                        #Necessario pous com um tamanho elevado de features as combinações possiveis são MUITA
                            #Nao precisamos explorar mais que s
        if done[c] > s: 
            break

        f = []
        for x in ff:  # transforma a tupla com o modelo em uma lista
            f.insert(len(f), x)
        auc, accmedia, preds, probs, f1, f1w = select_features_platelabel(
            df, f, params, nfolds=N_FOLDS, f1i=True)  # Chama a funcao central de treinamento p/ modelo f
        done[c] += 1
        res.append(np.mean(auc))
        exit_stat.write("%s;%f;%f;%f;%s;%s;%s;%s\n" %
                        (str(f), np.mean(auc),np.mean(f1),np.mean(f1w),auc,f1,f1w,accmedia))

        exit_outp.write("%s" % str(f))
        for p in preds:
            exit_outp.write(';%d' % p)
        for p in probs:
            exit_outp.write(';%f' % p)
        exit_outp.write('\n')
    return(res)

In [6]:
def delete_last_lines(ifile):
    with open(ifile, "r+", encoding="utf-8") as file:

        # Move the pointer (similar to a cursor in a text editor) to the end of the file
        file.seek(0, os.SEEK_END)

        # This code means the following code skips the very last character in the file -
        # i.e. in the case the last line is null we delete the last line
        # and the penultimate one
        pos = file.tell() - 1

        # Read each character in the file one at a time from the penultimate
        # character going backwards, searching for a newline character
        # If we find a new line, exit the search
        while pos > 0 and file.read(1) != "\n":
            pos -= 1
            file.seek(pos, os.SEEK_SET)

        # So long as we're not at the start of the file, delete all the characters ahead
        # of this position
        if pos > 0:
            file.seek(pos, os.SEEK_SET)
            file.truncate()

In [7]:
# Recebe UM modelo/parametros de decision tree e retorna scores, predições
# Retorna: aucs por fold, média de acuracia nos folds,
def select_features_platelabel(df, features, params, nfolds, f1i=False):  
    #

    X = df[features].values
    y = df[label_column_name].values
    predList = np.zeros(len(df))
    probList = np.zeros(len(df))

    cv = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=1)
    foldNum = 0
    a = []
    b = []
    c = []
    d = []
    for (train, val) in cv.split(X, y):
        #print(np.sum(y[train]),np.sum(y[val],len(y))
        foldNum = foldNum + 1

        # Modelo arvore
        classifier = DecisionTreeClassifier(class_weight='balanced', 
                                            max_depth=params['max_depth'], 
                                            min_samples_leaf=params['min_samples_leaf'],
                                            min_samples_split=params['min_samples_split'], 
                                            min_impurity_decrease=params['min_impurity_decrease'], 
                                            criterion=params['criterion'])
        classifier = classifier.fit(X[train], y[train])
        probas_ = classifier.predict_proba(X[val])
        
        #Extraimos[0] pois predict_proba retorna uma coluna pra cada classe, pegamos as probabilidades da 0
        probas = [probas_[x][0] for x in range(len(probas_))]

        pred = classifier.predict(X[val])
        area1 = roc_auc_score(y[val], probas_[:, 1])
        area2 = accuracy_score(y[val], pred)  # guarda acuracia

        #print('b',np.sum(y[val]),np.sum(pred),len(y[val]))
        f1 = f1_score(y[val],pred, average='binary')
        #print('w',np.sum(y[val]),np.sum(pred),len(y[val]))
        f1w = f1_score(y[val],pred, average='weighted')
        
        #a guarda o AUC score de cada fold
        a.insert(len(a), area1)
        #b guarda a acuraria score de cada fold
        b.insert(len(b), area2)
        #c guarda o F1-binary score de cada fold
        c.insert(len(c),f1)
        #d guarda o F1-weighted score de cada fold
        d.insert(len(d),f1w)

        for j in range(len(val)):
            #Como cada instancia em df vai estar no conjunto de val em algum fold,
            #predlist armazena as predições para todos eles em seus respectivos folds
            #probLIst armazena as probabilidades para todos eles em seus respectivos folds
            
            #val[i] contem o indice em X da i-esima instancia atualmente na validação
            #Assim se a instancia indice 3 do dataframe é o primeiro elemento no conjunto de validação atual,
            #  estamos fazendo predList[3] = pred[0] já que pred é indexado na ordem de val
            predList[val[j]] = pred[j]
            probList[val[j]] = probas[j]

    if f1i:
        return a, np.mean(b), predList, probList,c,d
    return a, np.mean(b), predList, probList

In [8]:
#df = pd.read_csv(TABLES_PATH + 'totalcx7.csv')
df = pd.read_pickle("./dfSourceFeatures28Aug2021.pkl")

In [9]:
print(df.shape)
print(df.describe())
print(df.info())
df.head(5)

(12045, 13)
        desinformacao_label  \
count                 12045   
unique                    2   
top                       0   
freq                  10000   

                                                      url    subdomain_ip  \
count                                               12045           11964   
unique                                              12023             244   
top     https://www.progresso.com.br/missao-visao-valo...  204.199.44.211   
freq                                                    2            2319   

       subdomain_ip_cc subdomain_ip_is_brazil subdomain_ip_is_us  \
count            11964                  11964              11964   
unique              12                      2                  2   
top                 US                  False               True   
freq              6759                   6874               6759   

        subdomain_ip_latitude  subdomain_ip_longitude subdomain_as_n  \
count                11964.00    

,desinformacao_label,url,subdomain_ip,subdomain_ip_cc,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_as_n,subdomain_as_cc,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count
0,0,https://g1.globo.com/politica/noticia/2021/07/...,186.192.81.31,BR,True,False,-22.87,-42.35,28604,BR,True,11,13
1,0,https://www.correio24horas.com.br/noticia/nid/...,204.199.44.211,BR,True,False,-19.92,-43.95,3549,US,False,12,3
2,0,https://www.gazetadopovo.com.br/ideias/capital...,2600:9000:208f:6800:6:45ad:3580:93a1,US,False,True,47.6,-122.33,16509,US,True,9,5
3,1,https://agoranoticiasbrasil.com.br/peritos-da-...,2606:4700:3035::ac43:86e4,US,False,True,37.78,-122.42,13335,US,True,9,1
4,0,https://g1.globo.com/economia/imposto-de-renda...,186.192.81.31,BR,True,False,-22.87,-42.35,28604,BR,True,11,13


In [10]:
df = df.dropna(axis=0)

In [11]:
print(df.shape)

(11964, 13)


In [12]:
df.columns

Index(['desinformacao_label', 'url', 'subdomain_ip', 'subdomain_ip_cc',
       'subdomain_ip_is_brazil', 'subdomain_ip_is_us', 'subdomain_ip_latitude',
       'subdomain_ip_longitude', 'subdomain_as_n', 'subdomain_as_cc',
       'subdomain_ipcc_equal_ascc', 'domain_route_hops',
       'domain_dns_caa_txt_count'],
      dtype='object')

In [13]:
df['desinformacao_label']= df['desinformacao_label'].astype('bool')
df['subdomain_ip_cc']= df['subdomain_ip_cc'].astype('category')
df['subdomain_ip_is_brazil']= df['subdomain_ip_is_brazil'].astype('bool')
df['subdomain_ip_is_us']= df['subdomain_ip_is_us'].astype('bool')
df['subdomain_ip_latitude']= df['subdomain_ip_latitude'].astype('float')
df['subdomain_ip_longitude']= df['subdomain_ip_longitude'].astype('float')
df['subdomain_as_n']= df['subdomain_as_n'].astype('category')
df['subdomain_as_cc']= df['subdomain_as_cc'].astype('category')
df['subdomain_ipcc_equal_ascc']= df['subdomain_ipcc_equal_ascc'].astype('bool')
df['domain_route_hops']= df['domain_route_hops'].astype('int')
df['domain_dns_caa_txt_count']= df['domain_dns_caa_txt_count'].astype('int')

In [14]:
label_column_name = 'desinformacao_label'
numericalFeatures = ['subdomain_ip_latitude', 'subdomain_ip_longitude','domain_route_hops','domain_dns_caa_txt_count']
categorialFeatures = ['subdomain_ip_cc','subdomain_ip_is_brazil','subdomain_ip_is_us', 
                      'subdomain_as_cc', 'subdomain_ipcc_equal_ascc']
nonFeatures = ['url','subdomain', 'subdomain_ip','subdomain_as_n' ]



columns = list(df.columns)


unwanted_columns = [label_column_name]+nonFeatures + categorialFeatures 
features_columns = [
    item for item in columns if item not in unwanted_columns
                   ]
print(len(df), np.sum(df[label_column_name]))
print(len(features_columns))

11964 1964
4


In [15]:
global done
global griddone
global predone
global queue_finished
queue_finished = 0
predone = 0
global s
#s = 10000
s = 1000
totalmodels = 0
combs = []
done = []
griddone = []

print('Creating Feature Combinations')
numeroMaximoDeFeatures = len(features_columns)
for c in range(0,numeroMaximoDeFeatures+1 ):
    # print('\t Size:%d'%c)
    if c == 0:
        combs.append([])
    else:
        combs.append(list(set(random_combinations(features_columns, c, s))))
    done.append(0)
    griddone.append(0)
    totalmodels += len(combs[-1])


Creating Feature Combinations


In [23]:
combs[2]

[('subdomain_ip_latitude', 'domain_route_hops'),
 ('subdomain_ip_latitude', 'domain_dns_caa_txt_count'),
 ('domain_route_hops', 'domain_dns_caa_txt_count'),
 ('subdomain_ip_longitude', 'domain_route_hops'),
 ('subdomain_ip_longitude', 'domain_dns_caa_txt_count'),
 ('subdomain_ip_latitude', 'subdomain_ip_longitude')]

In [16]:
def run_mmpool(c):
    sys.stdout.write("Starting MM size %d\n" % c)
    sys.stdout.flush()

    comb = combs[c]
    
    exit1 = open(MODELS_PATH + 'MultipleModels_DecisionTrees/' + TASK_NAME +
                 '-size%d-result.csv' % c, 'a+')
    exit2 = open(MODELS_PATH + 'MultipleModels_DecisionTrees/' + TASK_NAME +
                 '-size%d-preds.csv' % c, 'a+')

    a = eval_panel_platelabel(df, comb, c, exit1, exit2)

    global queue_finished
    queue_finished += 1
    exit1.close()
    exit2.close()
    return(a)


In [17]:
if 1==1:
    print('Creating Directories')
    if (not os.path.isdir(MODELS_PATH + 'MultipleModels_DecisionTrees')):
        os.mkdir(MODELS_PATH + 'MultipleModels_DecisionTrees')

    #for c in range(1, numeroMaximoDeFeatures+1):
    #Deletamos a ultima linha pro caso de ter rolado uma execução parcial
    for c in range(1, 5):
        if os.path.isfile('MultipleModels_DecisionTrees/' + TASK_NAME + '-size%d-result.csv' % c):
            delete_last_lines('MultipleModels_DecisionTrees/' + TASK_NAME +
                              '-size%d-result.csv' % c)
            delete_last_lines('MultipleModels_DecisionTrees/' + TASK_NAME +
                              '-size%d-preds.csv' % c)
    results = []
    for c in range(1,5):
        res = run_mmpool(c)
        results.append(res)
        
        
#     pool = Pool(processes=10)
#     #results = pool.map(run_mmpool, list(range(1, numeroMaximoDeFeatures+1)))
#     results = pool.map(run_mmpool, list(range(1, 3)))
#     time.sleep(10)
#     pool.join()

for i in range(0, len(results)):
    print(np.max(results[i]))

Creating Directories
Starting MM size 1
Starting MM size 2ess: 100.000% (4/4) [Elapsed: 0s | Predicted 0s | Avg: 0s]ed 638s | Avg: 0s]
Starting MM size 3ess: 100.000% (6/6) [Elapsed: 0s | Predicted 0s | Avg: 0s]ted 1013s | Avg: 0s]
Starting MM size 4ess: 100.000% (4/4) [Elapsed: 0s | Predicted 0s | Avg: 0s]ed 709s | Avg: 0s]
0.8386126651996676ess: 100.000% (1/1) [Elapsed: 0s | Predicted 0s | Avg: 0s]ed 212s | Avg: 0s]
0.8100053610375448
0.9870822158825362
0.9870822158825362
